In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [3]:
btc_raw_df = pd.read_parquet("/home/giujorge/Documents/Projets/bot_strategies/data/raw/BTC-USDT.csv.parquet.gzip")
btc_raw_df

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-18 14:09:00,39423.05,39423.06,39377.05,39385.62,22.37338000,1650290999999,881512.83100580,758,6.59638000,259885.09290650,0
2022-04-18 14:10:00,39385.61,39401.54,39370.24,39400.00,29.43069000,1650291059999,1159033.25545720,788,14.55146000,573044.31840130,0
2022-04-18 14:11:00,39400.00,39444.80,39399.83,39442.55,25.65637000,1650291119999,1011437.53518520,720,13.72377000,541044.94403500,0


In [4]:
btc_raw_df['EMA200'] = ta.trend.ema_indicator(btc_raw_df.close, window=200)

In [5]:
btc_raw_df['wf_top_bool'] = np.where(
                                        btc_raw_df['high'] == btc_raw_df['high'].rolling(9, center=True).max(), True, False
                                        )


In [7]:
btc_raw_df['wf_top'] = np.where(
                                        btc_raw_df['high'] == btc_raw_df['high'].rolling(9, center=True).max(), btc_raw_df['high'], None
                                        )


In [10]:
btc_raw_df['wf_top'] = btc_raw_df['wf_top'].ffill()

In [12]:
btc_raw_df.dropna(inplace=True)

In [14]:
btc_raw_df['buy'] = np.where((btc_raw_df.close > btc_raw_df.wf_top) & (btc_raw_df.close > btc_raw_df.EMA200), 1, 0)

In [16]:
btc_raw_df['SL'] = np.where(btc_raw_df.buy == 1, btc_raw_df.close - (btc_raw_df.close - btc_raw_df.low), 0)

In [17]:
btc_raw_df['TP'] = np.where(btc_raw_df.buy == 1, btc_raw_df.close + (btc_raw_df.close - btc_raw_df.low) * 1.5 ,0)

In [19]:
buys, sells = [], []

for i in range(len(btc_raw_df)):
    if btc_raw_df.buy.iloc[i]:
        buys.append(btc_raw_df.iloc[i].name)
        for j in range(len(btc_raw_df) - i):
            if btc_raw_df.TP.iloc[i] < btc_raw_df.close.iloc[i + j] or \
                btc_raw_df.SL.iloc[i] > btc_raw_df.close.iloc[i + j]:
                sells.append(btc_raw_df.iloc[i + j].name)
                break

In [22]:
frame = pd.DataFrame([buys, sells]).T.dropna()

In [23]:
frame.columns = ['Buys', 'Sells']

In [25]:
actualtrade = frame[frame.Buys > frame.Sells.shift(1)]

In [33]:
profits = (btc_raw_df.loc[actualtrade.Sells].close.values - btc_raw_df.loc[actualtrade.Buys].close.values)  / btc_raw_df.loc[actualtrade.Buys].close.values

In [34]:
profits

array([ 0.00696689,  0.00933639,  0.00358465, ...,  0.0018015 ,
       -0.00150843, -0.00340844])

In [35]:
profits.mean()

0.0003413509483364847

In [36]:
(profits + 1).prod()

1004168.0034921435

In [37]:
(btc_raw_df.close.pct_change() +1).cumprod()

timestamp
2017-08-17 07:19:00         NaN
2017-08-17 07:20:00    1.000000
2017-08-17 07:21:00    1.000000
2017-08-17 07:22:00    1.000000
2017-08-17 07:23:00    1.000000
                         ...   
2022-04-18 14:09:00    9.124180
2022-04-18 14:10:00    9.127512
2022-04-18 14:11:00    9.137369
2022-04-18 14:12:00    9.140941
2022-04-18 14:13:00    9.141298
Name: close, Length: 2447134, dtype: float64